Random Forest Classifier 


Raw pixel values of the images without extracting features

two empty lists, images and labels, which will be used to store the flattened images and their corresponding labels (0 for cats and 1 for dogs), respectively.

In [24]:
import cv2
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
from skimage import io
import shutil


In [29]:
# Paths to the directories containing Cat and Dog folders
base_path = "/Users/blanchepenafiel/Desktop/Dataset"
train_path= os.path.join(base_path, "Train")
cat_path = os.path.join(base_path, "Train", "Cat")
dog_path = os.path.join(base_path, "Train", "Dog")
cat_val_path=os.path.join(base_path, "Val", "Cat")
dog_val_path=os.path.join(base_path, "Val", "Dog")

In [20]:

# Function to check if an image is corrupted
def is_image_corrupted(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            return True
        return False
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return True

# Function to create a subset of non-corrupted images
def create_image_subset(input_folder, output_folder, subset_size):
    valid_images = []
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg"):
            image_path = os.path.join(input_folder, filename)
            if not is_image_corrupted(image_path):
                valid_images.append(image_path)
                if len(valid_images) >= subset_size:
                    break
    
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Copy valid images to output folder
    for image_path in valid_images:
        shutil.copy(image_path, output_folder)

# Define the subset size
subset_size = 5000

# Create subsets for cats and dogs
create_image_subset(cat_path, os.path.join(train_path, "train_cat_subset"), subset_size)
create_image_subset(dog_path, os.path.join(train_path, "train_dog_subset"), subset_size)

print("Subsets created successfully.")





Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 226 extraneous bytes before marker 0xd9


Subsets created successfully.


In [23]:


# Function to preprocess an image
def preprocess_image(image_path, target_size=(224, 224)):
    try:
        # Load image
        img = cv2.imread(image_path)
        if img is None:
            raise OSError("Failed to read image")
        
        # Resize image
        resized_img = cv2.resize(img, target_size)
        
        # Normalize image
        normalized_img = resized_img / 255.0
        
        # Flatten image
        flattened_img = normalized_img.flatten()
        
        return flattened_img
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# Function to preprocess all images in a folder
def preprocess_images_in_folder(folder_path, target_size=(224, 224)):
    preprocessed_images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            preprocessed_image = preprocess_image(image_path, target_size)
            if preprocessed_image is not None:
                preprocessed_images.append(preprocessed_image)
    return np.array(preprocessed_images)

# Paths to the directories containing cat and dog subsets
base_path = "/Users/blanchepenafiel/Desktop/Dataset"
train_path = os.path.join(base_path, "Train")
cat_subset_path = os.path.join(train_path, "train_cat_subset")
dog_subset_path = os.path.join(train_path, "train_dog_subset")

# Preprocess cat images
cat_images = preprocess_images_in_folder(cat_subset_path)

# Preprocess dog images
dog_images = preprocess_images_in_folder(dog_subset_path)

print("Images preprocessed successfully.")


Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 226 extraneous bytes before marker 0xd9


Images preprocessed successfully.


In [25]:
# Concatenate cat and dog images and create corresponding labels
X_train = np.concatenate([cat_images, dog_images])
y_train = np.concatenate([np.zeros(len(cat_images)), np.ones(len(dog_images))])




In [26]:
# Create and train the random forest model
rf1 = RandomForestClassifier(n_estimators=100, random_state=42)
rf1.fit(X_train, y_train)

print("Random forest model trained successfully.")

Random forest model trained successfully.


In [31]:
def load_images(folder_path, target_size=(224, 224, 3)):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):
            image_path = os.path.join(folder_path, filename)
            try:
                # Load image
                image = cv2.imread(image_path)
                if image is None:
                    raise OSError("Failed to read image")
                # Resize image to a fixed size
                resized_image = cv2.resize(image, (target_size[1], target_size[0]))
                # Normalize image
                normalized_image = resized_image / 255.0  # Scale pixel values to range [0, 1]
                # Append normalized image to the list
                images.append(normalized_image)
            except Exception as e:
                print(f"Error processing image {image_path}: {e}. Skipping...")
    return np.array(images)


In [34]:
# Load validation images
cat_val_images = load_images(cat_val_path)
dog_val_images = load_images(dog_val_path)

# Flatten and normalize images
cat_val_flat = cat_val_images.reshape(len(cat_val_images), -1)
dog_val_flat = dog_val_images.reshape(len(dog_val_images), -1)

# Concatenate cat and dog validation images
X_val = np.concatenate([cat_val_flat, dog_val_flat])

# Create corresponding labels for validation data
y_val = np.concatenate([np.zeros(len(cat_val_images)), np.ones(len(dog_val_images))])

# Evaluate the trained model on the validation data
accuracy = rf1.score(X_val, y_val)
print("Validation accuracy:", accuracy)


Validation accuracy: 0.6418884982420894
